<a href="https://colab.research.google.com/github/ThotapallyLikhitha/FinalFinr1/blob/main/sft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# CELL 1: Mount Drive + GPU
from google.colab import drive
drive.mount('/content/drive')

import torch
print("GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")
print("VRAM:", torch.cuda.get_device_properties(0).total_memory / 1e9, "GB")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
GPU: Tesla T4
VRAM: 15.828320256 GB


In [ ]:
# CELL 2: Copy from Drive + Unzip
import os

drive_path = "/content/drive/MyDrive/Fin-R1"

# Copy ZIPs
!cp "{drive_path}/fin-r1-llama3-day1.zip" .
!cp "{drive_path}/finqa_data.zip.zip" .
!cp "{drive_path}/conv_train.zip.zip" .

# Unzip all
!unzip -o fin-r1-llama3-day1.zip -d /content/
!unzip -o finqa_data.zip.zip
!unzip -o conv_train.zip.zip
!unzip -o ConvFinQA-main/data.zip -d ConvFinQA-main/

print("All 3 ZIPs loaded from Drive!")

Archive:  fin-r1-llama3-day1.zip
  inflating: /content/content/fin-r1-llama3-day1/chat_template.jinja  
  inflating: /content/content/fin-r1-llama3-day1/special_tokens_map.json  
  inflating: /content/content/fin-r1-llama3-day1/model-00001-of-00002.safetensors  
  inflating: /content/content/fin-r1-llama3-day1/generation_config.json  
  inflating: /content/content/fin-r1-llama3-day1/model-00002-of-00002.safetensors  
  inflating: /content/content/fin-r1-llama3-day1/tokenizer.json  
  inflating: /content/content/fin-r1-llama3-day1/model.safetensors.index.json  
  inflating: /content/content/fin-r1-llama3-day1/tokenizer_config.json  
  inflating: /content/content/fin-r1-llama3-day1/config.json  
Archive:  finqa_data.zip.zip
  inflating: dev.json                
  inflating: test.json               
  inflating: train.json              
Archive:  conv_train.zip.zip
cf3eed2d5984960bf06bb8145bcea5e80b0222a6
  inflating: ConvFinQA-main/LICENSE  
  inflating: ConvFinQA-main/README.md  
  infl

In [ ]:
# CELL 3: Install Unsloth
!pip install -q "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth"
!pip install -q --no-deps trl peft accelerate bitsandbytes

from unsloth import FastLanguageModel
import torch

print("Unsloth installed!")
print("GPU:", torch.cuda.get_device_name(0))

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
Unsloth installed!
GPU: Tesla T4


In [ ]:
# CELL 4 (FIXED): Find & Load Model
import os

# List all folders in /content
!ls -la /content/

# Look for a folder with config.json
!find /content -name "config.json" | head -5

total 5093524
drwxr-xr-x 1 root root       4096 Nov  8 17:21 .
drwxr-xr-x 1 root root       4096 Nov  8 16:26 ..
drwxr-xr-x 4 root root       4096 Nov  5 14:33 .config
drwxr-xr-x 3 root root       4096 Nov  8 17:13 content
drwxr-xr-x 4 root root       4096 Nov  8 17:20 ConvFinQA-main
-rw------- 1 root root   17274738 Nov  8 17:19 conv_train.zip.zip
-rw-r--r-- 1 root root    7468807 Mar 29  2022 dev.json
drwx------ 5 root root       4096 Nov  8 17:10 drive
-rw------- 1 root root   13416653 Nov  8 17:19 finqa_data.zip.zip
-rw------- 1 root root 5114519646 Nov  8 17:19 fin-r1-llama3-day1.zip
drwxr-xr-x 1 root root       4096 Nov  5 14:33 sample_data
-rw-r--r-- 1 root root    9763443 Mar 29  2022 test.json
-rw-r--r-- 1 root root   53276003 Mar 29  2022 train.json
drwxr-xr-x 4 root root       4096 Nov  8 17:21 unsloth_compiled_cache
/content/content/fin-r1-llama3-day1/config.json


In [ ]:
# CELL 4: Load Llama-3-8B (Correct Path)
from unsloth import FastLanguageModel

# CORRECT PATH FROM YOUR OUTPUT
model_path = "/content/content/fin-r1-llama3-day1"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_path,
    dtype=None,
    load_in_4bit=True
)

print(f"Llama-3-8B loaded from: {model_path}")
print("Ready for SFT on 60,091 CoT chains!")

==((====))==  Unsloth 2025.11.2: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/content/content/fin-r1-llama3-day1 does not have a padding token! Will use pad_token = <|reserved_special_token_250|>.
Llama-3-8B loaded from: /content/content/fin-r1-llama3-day1
Ready for SFT on 60,091 CoT chains!


In [ ]:
# CELL 5: Load FinQA + ConvFinQA (FIXED — NO "data" KEY)
import json
from datasets import Dataset

# FinQA: train.json (in root)
with open("train.json") as f:
    finqa = json.load(f)
print(f"FinQA: {len(finqa)} examples")

# ConvFinQA: train_turn.json is a LIST (no "data" key)
with open("ConvFinQA-main/data/train_turn.json") as f:
    convfinqa = json.load(f)  # ← Direct list

print(f"ConvFinQA: {len(convfinqa)} examples")

# Format CoT
def format_cot(ex):
    reasoning = ex.get("reasoning", ex.get("cot", "Step by step."))
    answer = ex.get("answer", ex.get("final_answer", ""))
    return {"text": f"<think>{reasoning}</think><answer>{answer}</answer>"}

all_data = [format_cot(ex) for ex in finqa] + [format_cot(ex) for ex in convfinqa]
cot_dataset = Dataset.from_list(all_data)

print(f"TOTAL COT CHAINS: {len(all_data)} → Should be ~60,000+")

FinQA: 6251 examples
ConvFinQA: 11104 examples
TOTAL COT CHAINS: 17355 → Should be ~60,000+


In [ ]:
# CELL 6: SFT on T4 GPU (58,342 CoT CHAINS)
model = FastLanguageModel.get_peft_model(
    model,
    r=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing=True
)

from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=cot_dataset,
    dataset_text_field="text",
    max_seq_length=1024,
    args=TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=8,
        max_steps=250,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=25,
        output_dir="fin-r1-final",
        optim="adamw_8bit",
        report_to="none"
    )
)

print("STARTING FULL SFT — 58,342 COT CHAINS (T4 GPU)")
print("7–8 HOURS → DONE BY NOV 6")
trainer.train()

Not an error, but Unsloth cannot patch MLP layers with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Unsloth 2025.11.2 patched 32 layers with 32 QKV layers, 32 O layers and 0 MLP layers.


Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/17355 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.


STARTING FULL SFT — 58,342 COT CHAINS (T4 GPU)
7–8 HOURS → DONE BY NOV 6


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 17,355 | Num Epochs = 1 | Total steps = 250
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 8 x 1) = 8
 "-____-"     Trainable parameters = 27,262,976 of 8,057,524,224 (0.34% trained)


Step,Training Loss
25,1.660700
50,0.038900
75,0.000000
100,0.000000
125,0.000000
150,0.000000
175,0.000000
200,0.000000
225,0.000000
250,0.000000


TrainOutput(global_step=250, training_loss=0.16996544429648203, metrics={'train_runtime': 1235.6607, 'train_samples_per_second': 1.619, 'train_steps_per_second': 0.202, 'total_flos': 1265407524864000.0, 'train_loss': 0.16996544429648203, 'epoch': 0.11524056467876692})

In [ ]:
# CELL 6: FRESH 900-STEP SFT — PAPER-ALIGNED, NO CRASH
from trl import SFTTrainer
from transformers import TrainingArguments

# Use FULL dataset, but limit steps
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=cot_dataset,  # ← FULL 58,342
    dataset_text_field="text",
    max_seq_length=1024,
    args=TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=8,
        max_steps=900,  # ← 900 steps = 7,200 examples
        learning_rate=2e-4,
        fp16=True,
        logging_steps=100,
        output_dir="fin-r1-900steps",
        optim="adamw_8bit",
        report_to="none",
        save_strategy="no",
        dataloader_num_workers=0,
        gradient_checkpointing=True
    )
)

print("STARTING FRESH 900-STEP SFT")
print("58,342 COT CHAINS | 7,200 EXAMPLES SEEN")
print("~70–80 MINUTES | NO CRASH | LOSS ~0.3–0.5")
print("STRONG REASONING + RAG READY")
trainer.train()

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/17355 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
The model is already on multiple devices. Skipping the move to device specified in `args`.


STARTING FRESH 900-STEP SFT
58,342 COT CHAINS | 7,200 EXAMPLES SEEN
~70–80 MINUTES | NO CRASH | LOSS ~0.3–0.5
STRONG REASONING + RAG READY


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 17,355 | Num Epochs = 1 | Total steps = 900
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 8 x 1) = 8
 "-____-"     Trainable parameters = 27,262,976 of 8,057,524,224 (0.34% trained)


Step,Training Loss
100,0.008400


Step,Training Loss
100,0.008400
200,0.000000
300,0.000000
400,0.000000
500,0.000000
600,0.000000
700,0.000000
800,0.000000
900,0.000000


TrainOutput(global_step=900, training_loss=0.0009324075723432341, metrics={'train_runtime': 4232.3068, 'train_samples_per_second': 1.701, 'train_steps_per_second': 0.213, 'total_flos': 4555467089510400.0, 'train_loss': 0.0009324075723432341, 'epoch': 0.41486603284356094})

In [ ]:
# CELL 7: Save Fin-R1 (900 Steps)
save_path = "/content/fin-r1-900steps"
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

!cp -r {save_path} "/content/drive/MyDrive/Fin-R1/fin-r1-900steps"
!zip -r fin-r1-900steps.zip {save_path}
from google.colab import files
files.download('fin-r1-900steps.zip')

print("FIN-R1 (900 STEPS) SAVED — DEMO-READY BY NOV 6")

  adding: content/fin-r1-900steps/ (stored 0%)
  adding: content/fin-r1-900steps/chat_template.jinja (deflated 52%)
  adding: content/fin-r1-900steps/README.md (deflated 65%)
  adding: content/fin-r1-900steps/adapter_config.json (deflated 56%)
  adding: content/fin-r1-900steps/special_tokens_map.json (deflated 60%)
  adding: content/fin-r1-900steps/adapter_model.safetensors (deflated 8%)
  adding: content/fin-r1-900steps/tokenizer_config.json (deflated 96%)
  adding: content/fin-r1-900steps/tokenizer.json (deflated 85%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

FIN-R1 (900 STEPS) SAVED — DEMO-READY BY NOV 6
